In [1]:
import glob,json
import pandas as pd
files = glob.glob("/home/suraj/Documents/Projects/job_market_data_lake/data/bronze/*.json")
all_jobs =[]
for file in files :
    with open(file, 'r') as f:
        data = json.load(f)
        all_jobs.extend(data['jobs'])

df = pd.DataFrame(all_jobs)

In [2]:
len(df)

58449

In [3]:
import numpy as np
df['ingestion_date']= None
# df.isnull().sum()            # count number of nulls on each columns
count=(df['salary_min']<=df['salary_max']).sum()
print(count)

27781


In [4]:
df = df.drop_duplicates('job_id', keep='last')

In [5]:
len(df)

14800

In [6]:
df.dtypes

job_id               object
title                   str
company                 str
city                    str
country                 str
salary_min          float64
salary_max          float64
salary_predicted        str
description             str
posted_date             str
ingestion_date       object
dtype: object

In [7]:
df['job_id']=df['job_id'].astype('string')
df['ingestion_date']=df['ingestion_date'].astype('string')
df.dtypes

job_id               string
title                   str
company                 str
city                    str
country                 str
salary_min          float64
salary_max          float64
salary_predicted        str
description             str
posted_date             str
ingestion_date       string
dtype: object

In [8]:
df.to_parquet('data.parquet', engine='pyarrow', compression='snappy')

In [9]:
df['title']=df['title'].str.lower()

In [11]:
df['salary_predicted'].unique()

<ArrowStringArray>
['0', '1']
Length: 2, dtype: str

In [ ]:
conditions =[
    df['title'].str.contains('manager') | df['title'].str.contains('senior'),
    df['title'].str.contains('data engineer'),
    df['title'].str.contains('data scientist') | df['title'].str.contains('data science'),
    df['title'].str.contains('analytics engineer'),
    df['title'].str.contains('analytics') | df['title'].str.contains('analyst'),
    df['title'].str.contains('platform'),
    df['title'].str.contains('develop'),
    df['title'].str.contains('engineer') | df['title'].str.contains('architect')
]
choices=[
    'project manager',
    'data engineer', 
    'data scientist', 
    'analytics engineer', 
    'data analytics', 
    'platform engineer',
    'software developer',
    'software engineer'
]
df['standardized_title']= np.select(conditions, choices, default="other")

In [ ]:
result = df.groupby("role").size().reset_index(name="count").sort_values(by="count", ascending=False)
import os
base_dir=  os.path.dirname(os.path.abspath(""))
result.to_csv('group_count.csv', index=False)

In [3]:
import sys
print(sys.executable)
print(type(sys.executable))

/home/suraj/de-venv/bin/python3.11
<class 'str'>


In [ ]:
import os
base_dir=  os.path.dirname(os.path.abspath(""))
print(base_dir)
os.path.join(base_dir, '')




/home/suraj/Documents


In [ ]:
import json, pandas as pd
path= "/home/suraj/Documents/Projects/job_market_data_lake/data/bronze/raw_jobs_2026_01_21.json"
with open(path, 'r') as f:
    data = json.load(f)
    df = pd.DataFrame(data['jobs'])
# df['description'] = df['description'].str.replace(r'\\.{5}', '', regex=True) 
df.head() 

,job_id,title,company,city,country,salary_min,salary_max,salary_predicted,description,posted_date
0,5591160809,Lead Data Engineer,Capital One,"Plano, Collin County",US,241508.97,241508.97,1,Lead Data Engineer Do you love building and pi...,2026-01-19T04:23:02Z
1,5565632348,Data Engineer,THE PRUITT GROUP LLC,"New Orleans, Orleans Parish",US,75195.24,75195.24,1,Job Description Job Description We are looking...,2026-01-02T22:55:06Z
2,5558900954,Data Engineer,Lancesoft INC,"San Bruno, San Mateo County",US,176800.00,187200.00,0,Job Description Job Description Job Title: Dat...,2025-12-29T00:42:56Z
3,5582315571,Data Engineer,The E Group,"Atlanta, Fulton County",US,74139.03,74139.03,1,Job Description Job Description Job Title: AI ...,2026-01-13T10:54:34Z
4,5565632556,Data Engineer,"Omnitech, Inc.","Sioux Falls, Minnehaha County",US,68726.22,68726.22,1,Job Description Job Description Description: P...,2026-01-02T22:55:55Z


In [13]:
import duckdb
duckdb.sql("select count(*) from read_parquet('~/Documents/Projects/job_market_data_lake/data/silver/jobs_cleaned_2026_01_21.parquet')").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         2117 │
└──────────────┘



In [2]:
from datetime import datetime
t= datetime.today().strftime("%Y_%m_%d")
print(type(t))

<class 'str'>
